In [1]:
import pandas as pd
train_df = pd.read_table('1. 실습용자료.txt',sep='|',encoding="cp949")
test_df = pd.read_table('2. 모델개발용자료.txt',sep='|',encoding="cp949")

In [ ]:
train_df.to_csv("1. 실습용자료.csv",encoding="utf-8-sig",index=False)
test_df.to_csv("2. 모델개발용자료.csv",encoding="utf-8-sig",index=False)

In [3]:
train_df.head(3)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매


In [4]:
test_df.head(3)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영


- text_obj, text_mthd, text_deal 3가지 텍스트 데이터를 기반으로 산업분류코드 예측하기

### 탐색적 데이터 분석(EDA)
- digit_1 : 대분류 21개
- digit_2 : 소분류 232개
- digit_3 : 세분류 495개
- '한국표준산업분류(10차)_국문.csv'의 표준산업분류 코드별 항목명 존재

실습용자료 불러오기, 열이름 수정, 열삭제

In [18]:
import pandas as pd
train_df = pd.read_csv("1. 실습용자료.csv")
train_df.columns = ["AI_id","대분류","중분류","소분류","text_obj","text_mthd","text_deal"]
#train_df.drop("AI_id",axis=1,inplace=True)

결측치 확인

In [6]:
train_df[train_df["text_obj"].isnull()]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal
46,id_0000047,I,55,551,NaN,NaN,펜션 운영업
231,id_0000232,G,46,465,NaN,"초음파세척기, 음식물처리기 도매업",NaN
305,id_0000306,Q,87,872,NaN,재가요양서비스기관,NaN
310,id_0000311,I,55,551,NaN,NaN,펜션운영업
353,id_0000354,G,46,464,NaN,향초 도매,NaN
...,...,...,...,...,...,...,...
999765,id_0999766,G,47,472,NaN,건강분말 구매후 판매,NaN
999800,id_0999801,C,22,222,NaN,"제품입고,숏트, 철,플라스틱",NaN
999848,id_0999849,F,42,424,NaN,토탈인테리어및 유리바닥재도소매,NaN
999913,id_0999914,J,59,592,NaN,녹음시설운영업,NaN


In [7]:
train_df[train_df["text_mthd"].isnull()]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal
21,id_0000022,R,91,911,당구,NaN,NaN
40,id_0000041,I,55,551,여관업,NaN,NaN
46,id_0000047,I,55,551,NaN,NaN,펜션 운영업
47,id_0000048,G,47,471,슈퍼마켓,NaN,NaN
58,id_0000059,G,46,467,"지붕,강판,집수리, 인테리어,태양광구조물",NaN,NaN
...,...,...,...,...,...,...,...
999975,id_0999976,C,25,259,기타임가공,NaN,NaN
999976,id_0999977,G,46,467,"고철,폐지 등",NaN,NaN
999986,id_0999987,I,56,561,음식점에서 육류 제공,NaN,NaN
999987,id_0999988,I,55,551,펜션운영업,NaN,NaN


In [8]:
train_df[train_df["text_deal"].isnull()]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal
10,id_0000011,C,10,102,멸치,"입고, 가공",NaN
21,id_0000022,R,91,911,당구,NaN,NaN
29,id_0000030,C,29,292,Resim,"재료구입, 세미앗세이",NaN
40,id_0000041,I,55,551,여관업,NaN,NaN
47,id_0000048,G,47,471,슈퍼마켓,NaN,NaN
...,...,...,...,...,...,...,...
999954,id_0999955,I,56,561,한식 음식점,NaN,NaN
999975,id_0999976,C,25,259,기타임가공,NaN,NaN
999976,id_0999977,G,46,467,"고철,폐지 등",NaN,NaN
999986,id_0999987,I,56,561,음식점에서 육류 제공,NaN,NaN


중복값 확인 : https://rfriend.tistory.com/266
- 대/중/소분류가 일치하더라도 text의 차이가 존재한다
- duplicated_df : 대/중/소분류에 중복이 존재하는 row
- 거의 모든 데이터가 대/중/소분류에 중복이 존재함
- 999064번 데이터 같은 경우는 text_obj과 text_mthd가 바꿔서 기입된 것이 아닌가 하는 의문

In [9]:
duplicated_df = train_df[train_df.duplicated(["대분류","중분류","소분류"])].sort_values(["대분류","중분류","소분류"])
duplicated_df

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal
5242,id_0005243,A,1,11,벼,모판생산,농가 출하
9375,id_0009376,A,1,11,사업장에서,오이.고추를재배,오이.고추를판매
16182,id_0016183,A,1,11,벼씨,발아,묘종
20631,id_0020632,A,1,11,NaN,NaN,벼재배
20987,id_0020988,A,1,11,벼,재배하여,벼재배
...,...,...,...,...,...,...,...
998903,id_0998904,S,96,969,영업장에서,일반인을대상으로,고속화물 수하물보관소
999001,id_0999002,S,96,969,세탁소에서,고객의의뢰를받다,"세탁,드라이크리닝"
999064,id_0999065,S,96,969,사망자대상으로,화장터에서,"화장,묘지분양관리"
999794,id_0999795,S,96,969,영업장에서,일반인을대상으로,고양이카페


표준사업분류 코드별 항목명 매핑해주기
- 앞 행의 데이터로 결측치 채우기 : https://seong6496.tistory.com/84
- 특정 열의 데이터 타입 변경하기 : https://hogni.tistory.com/70

In [3]:
code_df = pd.read_csv("한국표준산업분류(10차)_국문.csv")
code_df = code_df.fillna(method="ffill")[["대분류","대분류_항목명",
                                          "중분류","중분류_항목명",
                                         "소분류","소분류_항목명",
                                         "세분류","세분류_항목명",
                                         "세세분류","세세분류_항목명"]]
code_df = code_df.astype({'중분류': 'int',
                          '소분류': 'int',
                         '세분류': 'int',
                          '세세분류': 'int'})


중복값 확인
- 세분류의 111번과 소분류의 111번은 아예 다른 표준산업분류임

In [4]:
code_df[code_df["소분류"] == 111].head(3)

,대분류,대분류_항목명,중분류,중분류_항목명,소분류,소분류_항목명,세분류,세분류_항목명,세세분류,세세분류_항목명
89,C,제조업(10~34),11,음료 제조업,111,알코올 음료 제조업,1111,발효주 제조업,11111,탁주 및 약주 제조업
90,C,제조업(10~34),11,음료 제조업,111,알코올 음료 제조업,1111,발효주 제조업,11112,맥아 및 맥주 제조업
91,C,제조업(10~34),11,음료 제조업,111,알코올 음료 제조업,1111,발효주 제조업,11119,기타 발효주 제조업


In [5]:
code_df[code_df["세분류"] == 111].head(3)

,대분류,대분류_항목명,중분류,중분류_항목명,소분류,소분류_항목명,세분류,세분류_항목명,세세분류,세세분류_항목명
0,A,"농업, 임업 및 어업(01~03)",1,농업,11,작물 재배업,111,곡물 및 기타 식량작물 재배업,1110,곡물 및 기타 식량작물 재배업


표준산업분류와 실습자료 연결
- 대분류/중분류/소분류_항목명 연결

In [6]:
code_df = code_df[["대분류","대분류_항목명","중분류","중분류_항목명","소분류","소분류_항목명"]]
code_df = code_df.drop_duplicates(['대분류','중분류','소분류'], keep='first')

#merge_df = pd.merge(train_df,code_df,on=["대분류","중분류","소분류"],how="inner")
#merge_df.to_csv("실습용자료_한국표준산업분류.csv",encoding="utf-8-sig",index=False)

In [7]:
code_df

,대분류,대분류_항목명,중분류,중분류_항목명,소분류,소분류_항목명
0,A,"농업, 임업 및 어업(01~03)",1,농업,11,작물 재배업
10,A,"농업, 임업 및 어업(01~03)",1,농업,12,축산업
17,A,"농업, 임업 및 어업(01~03)",1,농업,13,작물재배 및 축산 복합농업
18,A,"농업, 임업 및 어업(01~03)",1,농업,14,작물재배 및 축산 관련 서비스업
21,A,"농업, 임업 및 어업(01~03)",1,농업,15,수렵 및 관련 서비스업
...,...,...,...,...,...,...
1181,S,"협회 및 단체, 수리 및 기타 개인 서비스업(94~96)",96,기타 개인 서비스업,969,그 외 기타 개인 서비스업
1192,T,가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동(97~98),97,가구 내 고용활동,970,가구 내 고용활동
1193,T,가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동(97~98),98,달리 분류되지 않은 자가 소비를 위한 가구의 재화 및 서비스 생산활동,981,자가 소비를 위한 가사 생산 활동
1194,T,가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동(97~98),98,달리 분류되지 않은 자가 소비를 위한 가구의 재화 및 서비스 생산활동,982,자가 소비를 위한 가사 서비스 활동


In [125]:
merge_df.head(3)

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,대분류_항목명,중분류_항목명,소분류_항목명
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,"협회 및 단체, 수리 및 기타 개인 서비스업(94~96)",개인 및 소비용품 수리업,자동차 및 모터사이클 수리업
1,id_0000017,S,95,952,사업장에서,고객요청으로,자동차세차업,"협회 및 단체, 수리 및 기타 개인 서비스업(94~96)",개인 및 소비용품 수리업,자동차 및 모터사이클 수리업
2,id_0000110,S,95,952,자동차,전문적으로 수리,자동차 전문수리업,"협회 및 단체, 수리 및 기타 개인 서비스업(94~96)",개인 및 소비용품 수리업,자동차 및 모터사이클 수리업


null 값 채우기

In [126]:
merge_df.fillna("",inplace=True)

한글 이외의 문자 처리
- 쉼표 -> 공백으로 변경하기
- 다중공백 -> 단일공백으로 변경하기 : https://m.blog.naver.com/wideeyed/221906671758

In [127]:
import re
def data_cleansing_text(df,col_lst):
    for col in col_lst:
        df[col] = df[col].apply(lambda x : re.sub(',', ' ', x))
        df[col] = df[col].apply(lambda x : re.sub(' +', ' ', x))
    return df

In [128]:
merge_df = data_cleansing_text(merge_df,["text_obj","text_mthd","text_deal"])

정규식을 이용한 한글 분류
- re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+') : 영어 및 숫자,기호 제외
- re.compile('[\u3131-\u3163\uac00-\ud7a3]+') : 영어 및 숫자,기호 등
- re.sub 로도 한글, 영어 분류가능
- null값(np.NaN)

In [129]:
# 한글만 추출하는 함수
def hangul_only(text):
    hangul = re.compile('([^ \u3131-\u3163\uac00-\ud7a3]+)')
    result = hangul.sub("",text)
    return result

# 한글만 제외하는 함수
def hangul_except(text):
    hangul = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')
    result = hangul.sub("",text)
    return result

In [130]:
# 3개 열에 대해서 각각 한글만 있는, 한글이 없는 열만 생성하기
df = merge_df
col_lst = ["text_obj","text_mthd","text_deal"]

for col in  col_lst:
    df[col+"_kr"] = df[col].apply(lambda x : hangul_only(x).strip()) # 한글만 있는 열 + 양쪽 공백제거
    df[col+"_nokr"] = df[col].apply(lambda x : hangul_except(x).strip()) # 한글이 없는 열 + 양쪽 공백제거

In [138]:
merge_df[["text_obj_kr","text_obj_nokr","text_obj"]].tail(5)

,text_obj_kr,text_obj_nokr,text_obj
999995,금속 테잎용원단,,금속 테잎용원단
999996,무연탄,,무연탄
999997,잎담배 판상엽,,잎담배 판상엽
999998,향료,,향료
999999,담배잎 담배잎줄기,,담배잎 담배잎줄기


한글 제외한 열 확인해보기
- 공백("")을 Null값으로 변경해서 공백만 존재하는 행 제외하고 확인하기
- 단순히 한글과 그 이외의 것으로 분류하면 안된다
- 자동차정비(3급) -> 자동차정비급 으로 분류된다
- 서비스 ( 자동차 -> 기록 시에 오류난 것

In [173]:
merge_df[merge_df["text_obj_nokr"] != ""].to_csv("실습용자료_nokr.csv",encoding="utf-8-sig",index=False)
# 한글이 제외된 텍스트를 열에 담아놓은 데이터프레임

특정 문자열만 포함한 행 출력하기 : https://zephyrus1111.tistory.com/46
- '㎡' 포함한 행만 출력한 결과, 대부분 G 47에 속함, 매장의 면적을 명시함

In [169]:
merge_df[merge_df["text_obj"].str.contains("㎡")][["대분류","중분류","소분류"]].value_counts()
                                                 #"대분류_항목명","중분류_항목명","소분류_항목명"]].value_counts()

대분류  중분류  소분류
G    47   471    3010
          472      20
          474       4
          476       4
          475       3
          478       3
I    56   561       2
G    46   463       1
N    75   752       1
R    91   912       1
S    96   961       1
dtype: int64

hanspell 맞춤법 검사기 이해
- 맞춤법 검사 후, 맞춤법 혹은 표준어가 의심되는 단어나 구절 key,value로 반환

In [32]:
from hanspell import spell_checker
from hanspell.constants import CheckResult

In [260]:
text = "안녕 하세요 저는 한국인 임니다 이 문장은 한ㅇ입니다"
result = spell_checker.check(text)

for key,value in result.words.items():
    print(key,value)

안녕하세요 2
저는 0
한국인입니다 1
이 0
문장은 0
한ㅇ입니다 0


## 클렌징 진행
- 문자 사이에 끼인 기호 처리하기, 정규표현식 이해 : https://data-newbie.tistory.com/210

In [63]:
import pandas as pd
train_df = pd.read_csv("1. 실습용자료.csv")
train_df.columns = ["AI_id","대분류","중분류","소분류","text_obj","text_mthd","text_deal"]
train_df.drop("AI_id",axis=1,inplace=True)

test_df = pd.read_csv("2. 모델개발용자료.csv")
test_df.columns = ["AI_id","대분류","중분류","소분류","text_obj","text_mthd","text_deal"]
test_df.drop("AI_id",axis=1,inplace=True)

In [67]:
# train_df와 test_df 결합
set_df = pd.concat([train_df,test_df],axis=0).reset_index(drop=True)
set_df.shape

(1100000, 6)

In [68]:
import re
from hanspell import spell_checker
from hanspell.constants import CheckResult
def data_cleansing_text(df,col_lst):
    for col in col_lst:
        # null값 채우기
        df[col] = df[col].fillna("")
        
        # 양쪽 공백 삭제
        df[col] = df[col].apply(lambda x : x.strip())
        
        # 영어 대문자 -> 소문자
        #df[col] = df[col].apply(lambda x : x.lower())
        
        #영어 문자 사이의 특수기호 삭제
        df[col] = df[col].str.replace(pat="(?<=[a-z])[/$](?=[a-z])",repl = "",regex=True)
        
        # ^ 삭제
        df[col] = df[col].str.replace("^","")
        
        # 특수기호 -> 공백으로
        #df[col] = df[col].apply(lambda x : re.sub('[^\w\s]', ' ', x))
        
        # 특정 특수기호 -> 공백으로
        df[col] = df[col].apply(lambda x : re.sub(r'[!@#$%&*<>?\|+-,.]', ' ', x))
        #% 또는 $ 또는 ^ 또는 * 또는 !을 없애주는 것이다.
        # ( ) 남기기
        
        # 숫자 지우기
        df[col] = df[col].apply(lambda x : re.sub(r'[0-9-]','',x))
        
        # 다중공백 -> 단일공백
        df[col] = df[col].apply(lambda x : re.sub(' +', ' ', x))
        
        # 한글 자음모음 삭제
        df[col] = df[col].apply(lambda x : re.sub('([ㄱ-ㅎㅏ-ㅣ]+)',"",x))
        
        # ㎡ 변환
        df[col] = df[col].apply(lambda x : re.sub('㎡',"제곱미터",x))
        
        # m2 변환
        df[col] = df[col].apply(lambda x : re.sub('m2',"제곱미터",x))
        
        # m2 변환
        df[col] = df[col].apply(lambda x : re.sub('M2',"제곱미터",x))
        # 맞춤법 검사기
        #df[col] = df[col].apply(lambda x : spell_checker.check(x).checked)
    return df

110만개 데이터 클렌징 진행
- 맞춤법 검사기는 통신 오류로 일단 배제
- text_obj, mthd, deal 컬럼을 합친 text_sum 열 생성하기
- apply 진행상황 확인 : https://doubly12f.tistory.com/56
- 실습용_모델개발용자료.csv : 맞춤법 검사를 제외한 기본적인 클렌징 train,test 데이터 셋에 대해 적용 행 제한 고려
- 실습용_모델개발용자료2.csv : 맞춤법 검사를 제외한 기본적인 클렌징 train,test 데이터 셋에 대해 적용 행 제한 고려

train_df, test_df 데이터 클렌징

In [71]:
train_df_clean = data_cleansing_text(train_df.copy(),["text_obj","text_mthd","text_deal"])
test_df_clean = data_cleansing_text(test_df.copy(),["text_obj","text_mthd","text_deal"])

train_df_clean["text_sum"] = set_df_clean["text_obj"] +" "+ set_df_clean["text_mthd"] +" "+ set_df_clean["text_deal"]
test_df_clean["text_sum"] = set_df_clean["text_obj"] +" "+ set_df_clean["text_mthd"] +" "+ set_df_clean["text_deal"]

<ipython-input-68-e7e8f1850fc4>:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace("^","")


In [70]:
#set_df_clean.to_csv("test.csv",encoding="utf-8-sig",index=False)

In [72]:
from tqdm import tqdm 
from hanspell import spell_checker
from hanspell.constants import CheckResult
tqdm.pandas()

train_df, test_df 띄어쓰기 및 맞춤법 보정

In [ ]:
train_df_clean["text_sum"] = train_df_clean["text_sum"].progress_apply(lambda x : spell_checker.check(x).checked)
test_df_clean["text_sum"] = test_df_clean["text_sum"].progress_apply(lambda x : spell_checker.check(x).checked)

 41%|████████████████████████████▏                                        | 409107/1000000 [3:19:54<3:17:31, 49.86it/s]

In [ ]:
#set_df_clean.to_csv("실습용_모델개발용자료_check.csv",encoding="utf-8-sig",index=False)

In [ ]:
train_preprocessed = train_df_clean.reset_index(drop=True)
test_preprocessed = test_df_clean.reset_index(drop=True)

In [ ]:
# column 데이터 타입 변경 int -> str
train_preprocessed = train_preprocessed.astype({'중분류': 'int',
                          '소분류': 'int'})

train_preprocessed = train_preprocessed.astype({'중분류': 'str',
                          '소분류': 'str'})

In [ ]:
#label 결합
train_preprocessed["label"] = ( train_preprocessed["대분류"] + train_preprocessed["중분류"]
                               + train_preprocessed["소분류"] )
test_preprocessed["label"] = ""

# 결합 이후 drop
for col in ["대분류","중분류","소분류"]:
    train_preprocessed.drop(col,axis=1,inplace=True)
    test_preprocessed.drop(col,axis=1,inplace=True)

In [ ]:
# 컬럼명 변경
train_preprocessed.rename(columns={'text_sum':'document'},inplace=True)
test_preprocessed.rename(columns={'text_sum':'document'},inplace=True)

In [60]:
train_preprocessed.to_csv("train_preprocesed_special.csv",encoding="utf-8-sig",index=False)
test_preprocessed.to_csv("test_preprocesed_special.csv",encoding="utf-8-sig",index=False)

### Imbalance 해소

In [18]:
train_preprocessed = pd.read_csv("train_preprocesed.csv")

In [20]:
train_preprocessed["대분류"] = train_preprocessed["label"].str[0]
train_preprocessed["중분류"] = train_preprocessed["label"].str[1:3]
train_preprocessed["소분류"] = train_preprocessed["label"].str[3:]

In [24]:
# Q1이 230 정도
train_preprocessed["label"].value_counts().describe()

count       225.000000
mean       4444.444444
std       11391.342413
min           1.000000
25%         230.000000
50%         865.000000
75%        3363.000000
max      123703.000000
Name: label, dtype: float64

In [29]:
train_preprocessed["label"].value_counts().to_csv("test.csv")

In [31]:
train_preprocessed[train_preprocessed["label"] == "C17171"]

,text_obj,text_mthd,text_deal,document,label,대분류,중분류,소분류
695,종이절단기,재단,종이,종이 절단기 재단 종이,C17171,C,17,171
11484,종이,절단 접합,봉투제조업,종이 절단 접합 봉투 제조업,C17171,C,17,171
17453,철판,전기판넬,생산,철판 전기패널 생산,C17171,C,17,171
20768,합성수지,판지,환풍기케이스종류제조,합성수지 판지 환풍기 케이스 종류 제조,C17171,C,17,171
23842,사업장에서,주문받아,엽서 제조,사업장에서 주문받아 엽서 제조,C17171,C,17,171
...,...,...,...,...,...,...,...,...
978141,영업장에서,주문자에의해,표면처리 코팅 인쇄제조,영업장에서 주문자에 의해 표면처리 코팅 인쇄 제조,C17171,C,17,171
981712,pp필름ugf pet,입고 가열압출,NaN,pp 필름 ugf pet 입고 가열 압출,C17171,C,17,171
982918,종이,재단,포장박스,종이 재단 포장박스,C17171,C,17,171
983330,감압지 감열지,절단 검사 포장,감압지원지,감압지 감열지 절단 검사 포장 감압 지원지,C17171,C,17,171


### 중복확인 및 label 데이터 교정

In [35]:
import pandas as pd
train_preprocessed = pd.read_csv("train_preprocesed.csv")

train_id = []
for i in range(1,1000001):
    num = str(i).zfill(7)
    train_id.append("id_" + num)
    
train_preprocessed["AI_id"] = train_id

대분류, 중분류, 소분류 부여

In [36]:
train_preprocessed["document"] = train_preprocessed["document"].apply(lambda x : x.strip())

In [10]:
train_preprocessed = train_preprocessed.drop_duplicates(["document","label"], keep='first', inplace=False, ignore_index=False).reset_index(drop=True)
# 100만행 -> 69만행

In [11]:
doc_dup = train_preprocessed[train_preprocessed.duplicated(["document"],keep=False)].sort_values("document")
doc_dup = doc_dup.reset_index(drop=True)

# 대분류, 중분류, 소분류 부여
doc_dup["대분류"] = doc_dup["label"].apply(lambda x : x[0])
doc_dup["중분류"] = doc_dup["label"].apply(lambda x : x[1:3] if len(x) == 6 else x[1]) # 4글자, 6글자인 경우 다르게 출력
doc_dup["소분류"] = doc_dup["label"].apply(lambda x : x[3:] if len(x) == 6 else x[-2:]) # 4글자, 6글자인 경우 다르게 출력

doc_dup # document만 중복인 것들 확인 : document가 같은데 label이 다른 것들

,text_obj,text_mthd,text_deal,document,label,AI_id,대분류,중분류,소분류
0,cctv,NaN,NaN,cctv,G47473,id_0568235,G,47,473
1,cctv,NaN,NaN,cctv,G46465,id_0525187,G,46,465
2,cctv설치,NaN,NaN,cctv 설치,S95951,id_0343689,S,95,951
3,cctv설치,NaN,NaN,cctv 설치,C34340,id_0257612,C,34,340
4,NaN,NaN,cctv설치,cctv 설치,G46465,id_0254560,G,46,465
...,...,...,...,...,...,...,...,...,...
8252,휴대폰케이스,NaN,NaN,휴대폰 케이스,G46464,id_0359187,G,46,464
8253,휴대폰케이스,NaN,NaN,휴대폰 케이스,C15151,id_0762020,C,15,151
8254,휴대폰케이스,NaN,NaN,휴대폰 케이스,G47474,id_0341768,G,47,474
8255,NaN,휴대폰판매,NaN,휴대폰 판매,G46465,id_0304052,G,46,465


In [12]:
doc_dup["document"].value_counts()

전자부품                            13
생활용품                            11
임가공                             10
의류                               9
공방                               9
                                ..
사업장에서 고객의 요청에 의해 소프트웨어 개발 공급     2
학원에서 초등학생 대상으로 영어교육              2
매장에서 일반인 대상 건강식품                 2
매장에서 소매 인테리어 소품                  2
노인복지를 위해 개인 간병인이 노인 환자 돌봄        2
Name: document, Length: 3869, dtype: int64

In [112]:
doc_dup[doc_dup["document"] == "의류"]

,text_obj,text_mthd,text_deal,document,label,AI_id,대분류,중분류,소분류
5830,의류,NaN,NaN,의류,G47474,id_0356785,G,47,474
5831,의류,NaN,NaN,의류,C14144,id_0637878,C,14,144
5832,의류,NaN,NaN,의류,G46464,id_0357820,G,46,464
5833,의류,NaN,NaN,의류,G47479,id_0363675,G,47,479
5834,의류,NaN,NaN,의류,G47478,id_0371762,G,47,478
5835,의류,NaN,NaN,의류,C14143,id_0589491,C,14,143
5836,의류,NaN,NaN,의류,C13134,id_0386634,C,13,134
5837,의류,NaN,NaN,의류,G47476,id_0541098,G,47,476
5838,의류,NaN,NaN,의류,C14141,id_0374912,C,14,141


In [113]:
train_preprocessed[(train_preprocessed["label"] == "G47474") |
                  (train_preprocessed["label"] == "G47478")]

,text_obj,text_mthd,text_deal,document,label,AI_id
12,매장에서,일반인 대상으로,의류소매,매장에서 일반인 대상으로 의류 소매,G47474,id_0000013
45,사업장에서,판매,갤러리 화랑,사업장에서 판매 갤러리 화랑,G47478,id_0000046
82,사업장에서,일반인대상,의류판매,사업장에서 일반인 대상 의류 판매,G47474,id_0000086
90,소매업,일반소비자에게,꽃 생화,소매업 일반 소비자에게 꽃 생화,G47478,id_0000094
94,안경점,일반소비자에게,안경소매,안경점 일반 소비자에게 안경 소매,G47478,id_0000098
...,...,...,...,...,...,...
638163,매장에서,일반인고객대상으로,화장품,매장에서 일반인 고객 대상으로 화장품,G47478,id_0999692
638272,영업점에서,일반소비자에게,여성의류,영업점에서 일반 소비자에게 여성의류,G47474,id_0999897
638280,일반인,화원,난및 꽃 소매,일반인 화원 난 및 꽃 소매,G47478,id_0999920
638283,사업장,일반인 대상,가발 판매,사업장 일반인 대상 가발 판매,G47474,id_0999924


### 산업분류 소분류 항목명과 연결
- 위쪽에서 전처리 code_df : 산업분류
- doc_dup : document가 겹치고 label 다른 것들
- doc_dup_merge : doc_dup과 code_df 연결

In [13]:
code_df = code_df.astype(str)

# join하기
doc_dup_merge = pd.merge(doc_dup,code_df[["소분류","소분류_항목명"]],on="소분류",how="inner")

In [14]:
# 소분류 항목명 텍스트 합치기
doc_dup_merge["document"] = doc_dup_merge["document"] +" "+ doc_dup_merge["소분류_항목명"]

# sorting
doc_dup_merge = doc_dup_merge.sort_values("document")

# drop column
doc_dup_merge = doc_dup_merge.drop(["대분류","중분류","소분류","소분류_항목명"],axis=1)

In [16]:
doc_dup_merge

,text_obj,text_mthd,text_deal,document,label,AI_id
0,cctv,NaN,NaN,cctv 가전제품 및 정보 통신장비 소매업,G47473,id_0568235
129,cctv,NaN,NaN,cctv 기계장비 및 관련 물품 도매업,G46465,id_0525187
130,NaN,NaN,cctv설치,cctv 설치 기계장비 및 관련 물품 도매업,G46465,id_0254560
372,cctv설치,NaN,NaN,cctv 설치 기타 상품 전문 소매업,G47478,id_0228929
334,cctv설치,NaN,NaN,cctv 설치 산업용 기계 및 장비 수리업,C34340,id_0257612
...,...,...,...,...,...,...
2828,휴대폰케이스,NaN,NaN,휴대폰 케이스 생활용품 도매업,G46464,id_0359187
3722,휴대폰케이스,NaN,NaN,"휴대폰 케이스 섬유, 의복, 신발 및 가죽제품 소매업",G47474,id_0341768
1512,휴대폰케이스,NaN,NaN,휴대폰 케이스 플라스틱 제품 제조업,C22222,id_0231290
128,NaN,휴대폰 판매,NaN,휴대폰 판매 가전제품 및 정보 통신장비 소매업,G47473,id_0004369


In [ ]:
#doc_dup_merge.to_csv("doc_dup_merge.csv",encoding="utf-8-sig",index=False)

train_preprocessed_dup : 기존의 중복이 없는 train_data와 document 중복, label 미중복을 해소한 data 결합

In [129]:
train_preprocessed_dup = pd.concat([train_preprocessed,doc_dup_merge],axis=0)

# 중복제거
train_preprocessed_dup = train_preprocessed_dup.drop_duplicates(["AI_id"], 
                                                                keep='last', 
                                                                ignore_index=False).reset_index(drop=True)

# 중복제거2
train_preprocessed_dup = train_preprocessed_dup.drop_duplicates(["document","label"], 
                                       keep='first',
                                       inplace=False, 
                                       ignore_index=False).reset_index(drop=True)

# Ai_id drop
train_preprocessed_dup = train_preprocessed_dup.drop("AI_id",axis=1)

In [132]:
train_preprocessed_dup.head(5)

,text_obj,text_mthd,text_deal,document,label
0,카센터에서,자동차부분정비,타이어오일교환,카센터에서 자동차 부분정비 타이어 오일 교환,S95952
1,상점내에서,일반인을 대상으로,채소 과일판매,상점 내에서 일반인을 대상으로 채소 과일 판매,G47472
2,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여 사업체에도 매 공업용 고무를 가지고 합성고무도 매,G46467
3,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서 일반 소비자에게 열쇠 잠금장치,G47475
4,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자의 위탁을 받아 취학 전 아동보육,Q87872


In [93]:
train_preprocessed_dup.to_csv("train_preprocessed_dup.csv",encoding="utf-8-sig",index=False)

In [137]:
train_preprocessed_dup["label"].value_counts()

I56561    61517
S96961    22806
G47474    21982
G47478    21834
I56562    20158
          ...  
B880          3
C12120        3
K65652        2
B551          1
B661          1
Name: label, Length: 225, dtype: int64

In [138]:
train_preprocessed_dup[train_preprocessed_dup["label"] == "B661"]

,text_obj,text_mthd,text_deal,document,label
33175,광산에서,채굴하여,철광생산,광산에서 채굴하여 철광 생산,B661


### 중복처리 없이 소분류 항목명만 add
- 소분류 항목명 add : doc_dup_merge

In [18]:
import pandas as pd
train_preprocessed = pd.read_csv("train_preprocesed.csv")

train_id = []
for i in range(1,1000001):
    num = str(i).zfill(7)
    train_id.append("id_" + num)
    
train_preprocessed["AI_id"] = train_id

In [19]:
train_preprocessed["document"] = train_preprocessed["document"].apply(lambda x : x.strip())

In [20]:
train_preprocessed.shape

(1000000, 6)

In [27]:
train_preprocessed_no_dup = pd.concat([train_preprocessed,doc_dup_merge],axis=0)

# 중복제거
train_preprocessed_no_dup = train_preprocessed_no_dup.drop_duplicates(["AI_id"], 
                                                                keep='last', 
                                                                ignore_index=False).reset_index(drop=True)
# 정렬
train_preprocessed_no_dup = train_preprocessed_no_dup.sort_values("AI_id").reset_index(drop=True)

In [29]:
train_preprocessed_no_dup

,text_obj,text_mthd,text_deal,document,label,AI_id
0,카센터에서,자동차부분정비,타이어오일교환,카센터에서 자동차 부분정비 타이어 오일 교환,S95952,id_0000001
1,상점내에서,일반인을 대상으로,채소 과일판매,상점 내에서 일반인을 대상으로 채소 과일 판매,G47472,id_0000002
2,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여 사업체에도 매 공업용 고무를 가지고 합성고무도 매,G46467,id_0000003
3,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서 일반 소비자에게 열쇠 잠금장치,G47475,id_0000004
4,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자의 위탁을 받아 취학 전 아동보육,Q87872,id_0000005
...,...,...,...,...,...,...
999995,제품입고,워싱,청바지워싱,제품 입고 워싱 청바지 워싱,C13134,id_0999996
999996,현장에서,고객의요청에의해,실내인테리어,현장에서 고객의 요청에 의해 실내 인테리어,F42424,id_0999997
999997,영업점에서,일반소비자에게,여성의류 판매,영업점에서 일반 소비자에게 여성의류 판매,G47474,id_0999998
999998,사업장에서,일반고객을대상으로,필라테스,사업장에서 일반 고객을 대상으로 필라테스,P85856,id_0999999


In [31]:
train_preprocessed_no_dup.to_csv("train_preprocessed_no_dup.csv",encoding="utf-8-sig",index=False)

# 제출된 답안 비교
- 답안 제출일자에 따라 다르게 예측되는 값들 확인해보기
- 0329 : 성능 90.43 / kobert / label 미제거 / 제출형식 준수
- 0330 : 성능 88.43 / kobert / label 제거 / 제출형식 준수
- 0331 : 성능 ? / klue roberta-base / label 미제거 / 제출형식 준수

In [3]:
import pandas as pd
predict_0329 = pd.read_csv("답안제출_0329.csv",encoding="utf-8-sig")
predict_0330 = pd.read_csv("답안제출_0330.csv",encoding="utf-8-sig")
predict_0331 = pd.read_csv("답안제출_0331.csv",encoding="utf-8-sig")

In [6]:
def make_label_column(df):
    df = df.astype(str)
    df["label"] = df["digit_1"] + df["digit_2"] + df["digit_3"]
    return df

predict_0329 = make_label_column(predict_0329)
predict_0330 = make_label_column(predict_0330)
predict_0331 = make_label_column(predict_0331)

In [14]:
# 29일자 결과물과 31일자 결과물 비교. 다른 label만
compare_0329_0331 = predict_0329.copy()
compare_0329_0331 = compare_0329_0331[predict_0331["label"] != predict_0329["label"]]

# label 비교
compare_0329_0331["label2"] = predict_0331[predict_0331["label"] != predict_0329["label"]]["label"]


In [17]:
compare_0329_0331.to_csv("compare_0329_0331.csv",encoding="utf-8-sig",index=False)

In [18]:
# 29일자 결과물과 30일자 결과물 비교. 다른 label만
compare_0329_0330 = predict_0329.copy()
compare_0329_0330 = compare_0329_0330[predict_0330["label"] != predict_0329["label"]]

# label 비교
compare_0329_0330["label2"] = predict_0330[predict_0330["label"] != predict_0329["label"]]["label"]


In [19]:
compare_0329_0330.to_csv("compare_0329_0330.csv",encoding="utf-8-sig",index=False)

In [21]:
compare_0329_0331

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,label,label2
3,id_000004,C,30,302,영업장에서,고객요구로,자동차튜닝,C30302,S95952
9,id_000010,C,25,259,"약품(화공), 미싱","완성품입고, 수선",nan,C25259,S95953
12,id_000013,C,16,162,목재,고객의 요구에 따라,무대장치,C16162,C32320
14,id_000015,M,72,729,산업공단조성시 의뢰받아,사무실에서,측량，토목설계,M72729,M72721
17,id_000018,G,47,472,상가에서,주문에 의해,"반찬,도시락",G47472,C10107
...,...,...,...,...,...,...,...,...,...
99972,id_099973,C,32,320,목재,접합 절단,목재깔판,C32320,C16162
99979,id_099980,N,75,759,업주들을대상으로,사무실에서,검사및회계사무지원,N75759,M71712
99985,id_099986,C,29,291,철,"구입, 기계절단",nan,C29291,C29292
99989,id_099990,G,47,472,사업장에서,일반소비자및 소매업자,계란소매,G47472,G46463


In [6]:
import pandas as pd
tt = pd.read_csv("kobert_submission_0331.csv").drop("Unnamed: 0",axis=1)

In [8]:
tt.to_csv("답안제출_0401.csv",encoding="utf-8-sig",index=False)

In [39]:
tt = pd.read_csv("doc_dup.csv")

tt

,text_obj,text_mthd,text_deal,document,label
0,0,입고 취부,NaN,0 입고 취부,C31311
1,00가정에서,방문안전확인,노인돌봄서비스제공,00가정에서 방문 안전 확인 노인 돌봄 서비스 제공,S94949
2,090ⅱ2m,수입검사 조립,NaN,090ⅱ2m 수입검사 조립,C28281
3,NaN,1 1 운동으로 통증재활 자세교정 일반교정운동,NaN,1 1 운동으로 통증재활 자세 교정 일반교정운동,P85856
4,1 2톤화물차를가지고,고객의요청으로,개별운송,1 2톤 화물차를 가지고 고객의 요청으로 개별 운송,H49493
...,...,...,...,...,...
630061,힐튼호텔,두발미용 서비스,머리자르기 염색,힐튼호텔 두발 미용 서비스 머리 자르기 염색,S96961
630062,힐튼호텔,일반소비자에게 소매,화원,힐튼호텔 일반 소비자에게 소매 화원,G47478
630063,힐티에서,다른업자에게 도매,전동공구 도매,힐티에서 다른 업자에게 도매 전동공구 도매,G46465
630064,심영암에서,NaN,점술서비스,힘 영암에서 점술 서비스,S96969
